# Global stress objective example

## Commented Program

What follows is a program spliced with comments.

In [1]:
using TopOpt, LinearAlgebra

### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 3.0 # filter radius

problems = Any[
    PointLoadCantilever(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
    HalfMBB(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
]
problem_names = ["Cantilever beam", "Half MBB beam", "L-beam", "Tie-beam"]

i = 1
println(problem_names[i])
problem = problems[i]

Cantilever beam


### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.001 # minimum density
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1
convcriteria = Nonconvex.KKTCriteria()
penalty = TopOpt.PowerPenalty(1.0)

PowerPenalty{Float64}(1.0)

### Define a finite element solver

In [4]:
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)

### Define **stress** objective
Notice that gradient is derived automatically by automatic differentiation (Zygote.jl)!

In [5]:
stress = TopOpt.MicroVonMisesStress(solver)
filter = if problem isa TopOptProblems.TieBeam
    identity
else
    DensityFilter(solver; rmin=rmin)
end
volfrac = TopOpt.Volume(problem, solver)

obj = x -> volfrac(filter(x))
constr = x -> norm(stress(filter(x)), 5) - 1.0
options = MMAOptions(; maxiter=2000, tol=Nonconvex.Tolerance(; kkt=1e-4))

MMAOptions{Float64, Tolerance{Float64, Float64, Float64, Float64}, Optim.Options{Float64, Nothing}}
  maxiter: Int64 2000
  outer_maxiter: Int64 100000000
  maxinner: Int64 10
  tol: Tolerance{Float64, Float64, Float64, Float64}
  s_init: Float64 0.5
  s_incr: Float64 1.2
  s_decr: Float64 0.7
  store_trace: Bool false
  show_trace: Bool false
  auto_scale: Bool false
  keep_best: Bool false
  dual_options: Optim.Options{Float64, Nothing}


### Define subproblem optimizer

In [6]:
x0 = fill(1.0, length(solver.vars))
optimizer = Optimizer(obj, constr, x0, MMA87(); options=options, convcriteria=convcriteria)

Internal error: encountered unexpected error in runtime:
TypeError(func=:<:, context="", expected=Type, got=TApprox<:NonconvexMMA.AbstractMMAApprox)
jl_type_error_rt at /buildworker/worker/package_linux64/build/src/rtutils.c:119
jl_type_error at /buildworker/worker/package_linux64/build/src/rtutils.c:127
jl_f_issubtype at /buildworker/worker/package_linux64/build/src/builtins.c:491
apply_type_nothrow at ./compiler/tfuncs.jl:1245
_builtin_nothrow at ./compiler/tfuncs.jl:1566
stmt_effect_free at ./compiler/ssair/queries.jl:36
check_effect_free! at ./compiler/ssair/inlining.jl:1128 [inlined]
process_simple! at ./compiler/ssair/inlining.jl:1185
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1358
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:80
jfptr_ssa_inlining_passNOT._12916.clone_1 at /opt/hostedtoolcache/julia/1.7.2/x64/lib/julia/sys.so (unknown line)
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2247 [inlined]
jl_apply_generic at /buildworker/worker/pack

### Define continuation SIMP optimizer

In [7]:
simp = SIMP(optimizer, solver, 3.0)

### Solve

In [8]:
result = simp(x0)

@show result.convstate
@show optimizer.workspace.iter
@show result.objval

result.convstate = NonconvexCore.ConvergenceState{Float64}
  Δx: Float64 0.10294031730572727
  Δf: Float64 7.351883508677215e-5
  relΔf: Float64 8.815834463725918e-5
  kkt_residual: Float64 8.950752582535489e-5
  ipopt_residual: Float64 8.950752582535489e-5
  infeas: Float64 8.895008393405845e-6
  x_converged: Bool false
  fabs_converged: Bool false
  frel_converged: Bool false
  kkt_converged: Bool true
  ipopt_converged: Bool true
  infeas_converged: Bool true
  f_increased: Bool true
  converged: Bool true

optimizer.workspace.iter = 1605
result.objval = 0.8339307391243166


0.8339307391243166

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(problem; topology = result.topology)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*